In [1]:
import islpy as isl
from islplot_support import get_set_list, get_umap_list, plot_usets, plot_umaps, print_before_after
from latex_op import display_latex, print_latex

# Classical Loop Transformations

### Setup AST generation infrastructure

In [8]:
def print_before_after(domain, schedule_original, schedule_new):
    context = isl.Set("{ : }")
    build = isl.AstBuild.from_context(context)
    schedule_original = schedule_original.intersect_domain(domain)
    schedule_new = schedule_new.intersect_domain(domain)
    print("<b>Before Transform:</b>")
    ast = build.node_from_schedule_map(schedule_original)
    print(ast)
    print("<b>After Transform:</b>")
    ast = build.node_from_schedule_map(schedule_new)
    print(ast)

## Loop Reversal

Loop reversal changes the direction in which elements of a loop are visited. After loop reversal, the previous first loop iteration is executed last and the previous last loop iteration is executed first.

**Benefits**:
- Can be used to shorten dependences

In [9]:
domain = isl.UnionSet("[n] -> {S[i] : 0 <= i < n}")
original = isl.UnionMap("{S[i] -> [i]}")
transformation = isl.UnionMap("{[i] -> [-i]}")

transformed = original.apply_range(transformation)
print_before_after(domain, original, transformed)

<b>Before Transform:</b>
{ iterator: { id: c0 }, init: { val: 0 }, cond: { op: lt, args: [ { id: c0 }, { id: n } ] }, inc: { val: 1 }, body: { user: { op: call, args: [ { id: S }, { id: c0 } ] } } }
<b>After Transform:</b>
{ iterator: { id: c0 }, init: { op: add, args: [ { op: minus, args: [ { id: n } ] }, { val: 1 } ] }, cond: { op: le, args: [ { id: c0 }, { val: 0 } ] }, inc: { val: 1 }, body: { user: { op: call, args: [ { id: S }, { op: minus, args: [ { id: c0 } ] } ] } } }


# Loop Fusion

After Loop fusion two statements that have previously been enumerated by different loops are
now enumerated by a single loop.

**Benefits:**
  - Improves data-locality


In [10]:
domain = isl.UnionSet("[n] -> {S[i] : 0 <= i <= n; T[i] : 0 <= i <= n}")
original = isl.UnionMap("{S[i] -> [0, i]; T[i] -> [1, i]}")
transformation = isl.UnionMap("{[0, i] -> [i,0]; [1, i] -> [i, 1]}")
transformed = original.apply_range(transformation)
print_before_after(domain, original, transformed)

<b>Before Transform:</b>
[ { iterator: { id: c1 }, init: { val: 0 }, cond: { op: le, args: [ { id: c1 }, { id: n } ] }, inc: { val: 1 }, body: { user: { op: call, args: [ { id: S }, { id: c1 } ] } } }, { iterator: { id: c1 }, init: { val: 0 }, cond: { op: le, args: [ { id: c1 }, { id: n } ] }, inc: { val: 1 }, body: { user: { op: call, args: [ { id: T }, { id: c1 } ] } } } ]
<b>After Transform:</b>
{ iterator: { id: c0 }, init: { val: 0 }, cond: { op: le, args: [ { id: c0 }, { id: n } ] }, inc: { val: 1 }, body: [ { user: { op: call, args: [ { id: S }, { id: c0 } ] } }, { user: { op: call, args: [ { id: T }, { id: c0 } ] } } ] }


# Loop Fission (Loop Distribution)

Loop fission takes two statements that have been originally executed in the same
loop and distributes them to two separate loops.

**Benefits:**
 - Reduces register pressure
 - Enables other transformations, i.e. SIMDization in case only one of
   the two statements in a loop body allows for parallel execution.

In [11]:
domain = isl.UnionSet("[n] -> {S[i] : 0 <= i <= n; T[i] : 0 <= i <= n}")
original = isl.UnionMap("{S[i] -> [i, 0]; T[i] -> [i, 1]}")
transformation = isl.UnionMap("{[i, 0] -> [0, i]; [i, 1] -> [1, i]}")

transformed = original.apply_range(transformation)
print_before_after(domain, original, transformed)

<b>Before Transform:</b>
{ iterator: { id: c0 }, init: { val: 0 }, cond: { op: le, args: [ { id: c0 }, { id: n } ] }, inc: { val: 1 }, body: [ { user: { op: call, args: [ { id: S }, { id: c0 } ] } }, { user: { op: call, args: [ { id: T }, { id: c0 } ] } } ] }
<b>After Transform:</b>
[ { iterator: { id: c1 }, init: { val: 0 }, cond: { op: le, args: [ { id: c1 }, { id: n } ] }, inc: { val: 1 }, body: { user: { op: call, args: [ { id: S }, { id: c1 } ] } } }, { iterator: { id: c1 }, init: { val: 0 }, cond: { op: le, args: [ { id: c1 }, { id: n } ] }, inc: { val: 1 }, body: { user: { op: call, args: [ { id: T }, { id: c1 } ] } } } ]


# Loop Interchange

In [12]:
domain = isl.UnionSet("[n,m] -> {S[i,j] : 0 <= i <= n and 0 <= j <= m }")
original = isl.UnionMap("{S[i,j] -> [i, j]}")
transformation = isl.UnionMap("{[i, j] -> [j, i]}")

transformed = original.apply_range(transformation)
print_before_after(domain, original, transformed)

<b>Before Transform:</b>
{ iterator: { id: c0 }, init: { val: 0 }, cond: { op: le, args: [ { id: c0 }, { id: n } ] }, inc: { val: 1 }, body: { iterator: { id: c1 }, init: { val: 0 }, cond: { op: le, args: [ { id: c1 }, { id: m } ] }, inc: { val: 1 }, body: { user: { op: call, args: [ { id: S }, { id: c0 }, { id: c1 } ] } } } }
<b>After Transform:</b>
{ iterator: { id: c0 }, init: { val: 0 }, cond: { op: le, args: [ { id: c0 }, { id: m } ] }, inc: { val: 1 }, body: { iterator: { id: c1 }, init: { val: 0 }, cond: { op: le, args: [ { id: c1 }, { id: n } ] }, inc: { val: 1 }, body: { user: { op: call, args: [ { id: S }, { id: c1 }, { id: c0 } ] } } } }


# Strip Mining

Strip mining partitions a single loop into chunks that are enumerated by two loops.
An outer loop enumerates the individual blocks, whereas the inner loop enumerates
the individual iterations that belong to each block.

**Benefits:**
 - Building block for loop tiling and unroll-and-jam.

In [13]:
domain = isl.UnionSet("{S[i] : 0 <= i < 1024 }")
original = isl.UnionMap("{S[i] -> [i]}")
transformation = isl.UnionMap("{[i] -> [floor(i/4), i % 4]}")

transformed = original.apply_range(transformation)
print_before_after(domain, original, transformed)

<b>Before Transform:</b>
{ iterator: { id: c0 }, init: { val: 0 }, cond: { op: le, args: [ { id: c0 }, { val: 1023 } ] }, inc: { val: 1 }, body: { user: { op: call, args: [ { id: S }, { id: c0 } ] } } }
<b>After Transform:</b>
{ iterator: { id: c0 }, init: { val: 0 }, cond: { op: le, args: [ { id: c0 }, { val: 255 } ] }, inc: { val: 1 }, body: { iterator: { id: c1 }, init: { val: 0 }, cond: { op: le, args: [ { id: c1 }, { val: 3 } ] }, inc: { val: 1 }, body: { user: { op: call, args: [ { id: S }, { op: add, args: [ { op: mul, args: [ { val: 4 }, { id: c0 } ] }, { id: c1 } ] } ] } } } }


# Loop Tiling

Loop tiling partitions the execution of a multi-dimensional loop into groups, the tiles.
First a set of outer loops enumerate all tiles that must be executed and for each tile
a set of inner loops, the point loops, enumerates the individual points of the tile.

**Benefits:**
 - Increased data-locality
 - More coarse-grained parallelism

In [14]:
domain = isl.UnionSet("{S[i,j] : 0 <= i,j < 1024 }")
original = isl.UnionMap("{S[i,j] -> [i,j]}")
transformation = isl.UnionMap("{[i,j] -> [floor(i/4), i % 4, floor(j/4), j % 4]}")

transformed = original.apply_range(transformation)
print_before_after(domain, original, transformed)

<b>Before Transform:</b>
{ iterator: { id: c0 }, init: { val: 0 }, cond: { op: le, args: [ { id: c0 }, { val: 1023 } ] }, inc: { val: 1 }, body: { iterator: { id: c1 }, init: { val: 0 }, cond: { op: le, args: [ { id: c1 }, { val: 1023 } ] }, inc: { val: 1 }, body: { user: { op: call, args: [ { id: S }, { id: c0 }, { id: c1 } ] } } } }
<b>After Transform:</b>
{ iterator: { id: c0 }, init: { val: 0 }, cond: { op: le, args: [ { id: c0 }, { val: 255 } ] }, inc: { val: 1 }, body: { iterator: { id: c1 }, init: { val: 0 }, cond: { op: le, args: [ { id: c1 }, { val: 3 } ] }, inc: { val: 1 }, body: { iterator: { id: c2 }, init: { val: 0 }, cond: { op: le, args: [ { id: c2 }, { val: 255 } ] }, inc: { val: 1 }, body: { iterator: { id: c3 }, init: { val: 0 }, cond: { op: le, args: [ { id: c3 }, { val: 3 } ] }, inc: { val: 1 }, body: { user: { op: call, args: [ { id: S }, { op: add, args: [ { op: mul, args: [ { val: 4 }, { id: c0 } ] }, { id: c1 } ] }, { op: add, args: [ { op: mul, args: [ { val: 4

# Unroll-and-jam

Unroll-and-jam is a combination of strip-mining of the outer loop into a
tile and point loop and then an interchange of the new point loop with
the innermost loop dimension.

**Benefits:**
 - Enables outer loop vectorization

In [15]:
domain = isl.UnionSet("{S[i,j] : 0 <= i,j < 1024 }")
original = isl.UnionMap("{S[i,j] -> [i,j]}")
transformation = isl.UnionMap("{[i,j] -> [floor(i/4), j, i % 4] }")

transformed = original.apply_range(transformation)
print_before_after(domain, original, transformed)

<b>Before Transform:</b>
{ iterator: { id: c0 }, init: { val: 0 }, cond: { op: le, args: [ { id: c0 }, { val: 1023 } ] }, inc: { val: 1 }, body: { iterator: { id: c1 }, init: { val: 0 }, cond: { op: le, args: [ { id: c1 }, { val: 1023 } ] }, inc: { val: 1 }, body: { user: { op: call, args: [ { id: S }, { id: c0 }, { id: c1 } ] } } } }
<b>After Transform:</b>
{ iterator: { id: c0 }, init: { val: 0 }, cond: { op: le, args: [ { id: c0 }, { val: 255 } ] }, inc: { val: 1 }, body: { iterator: { id: c1 }, init: { val: 0 }, cond: { op: le, args: [ { id: c1 }, { val: 1023 } ] }, inc: { val: 1 }, body: { iterator: { id: c2 }, init: { val: 0 }, cond: { op: le, args: [ { id: c2 }, { val: 3 } ] }, inc: { val: 1 }, body: { user: { op: call, args: [ { id: S }, { op: add, args: [ { op: mul, args: [ { val: 4 }, { id: c0 } ] }, { id: c2 } ] }, { id: c1 } ] } } } } }


# Skewing


In [17]:
domain = isl.UnionSet("[n] -> {S[i,j] : 0 <= i,j < n }")
original = isl.UnionMap("{S[i,j] -> [i,j]}")
transformation = isl.UnionMap("{[i,j] -> [i, i + j]}")

transformed = original.apply_range(transformation)
print_before_after(domain, original, transformed)

<b>Before Transform:</b>
{ iterator: { id: c0 }, init: { val: 0 }, cond: { op: lt, args: [ { id: c0 }, { id: n } ] }, inc: { val: 1 }, body: { iterator: { id: c1 }, init: { val: 0 }, cond: { op: lt, args: [ { id: c1 }, { id: n } ] }, inc: { val: 1 }, body: { user: { op: call, args: [ { id: S }, { id: c0 }, { id: c1 } ] } } } }
<b>After Transform:</b>
{ iterator: { id: c0 }, init: { val: 0 }, cond: { op: lt, args: [ { id: c0 }, { id: n } ] }, inc: { val: 1 }, body: { iterator: { id: c1 }, init: { id: c0 }, cond: { op: lt, args: [ { id: c1 }, { op: add, args: [ { id: n }, { id: c0 } ] } ] }, inc: { val: 1 }, body: { user: { op: call, args: [ { id: S }, { id: c0 }, { op: add, args: [ { op: minus, args: [ { id: c0 } ] }, { id: c1 } ] } ] } } } }
